In [1]:
import pandas as pd

# Paths
comp_path = "/Users/amarachukwuobi/Downloads/compustat_breach_exactmatch.csv"
fin_path  = "/Users/amarachukwuobi/Desktop/DataBreach/findata_ch_updated.xlsx"

# Read
comp = pd.read_csv(comp_path, low_memory=False)
fin  = pd.read_excel(fin_path, dtype={"cusip": str})

# Rename comp key to align with fin
comp = comp.rename(columns={"compustat_cusip": "cusip"})

# Clean keys
comp["cusip"] = comp["cusip"].astype(str).str.strip()
fin["cusip"]  = fin["cusip"].astype(str).str.strip()

comp["iymd"] = pd.to_numeric(comp["iymd"], errors="coerce").astype("Int64")
fin["iymd"]  = pd.to_numeric(fin["iymd"], errors="coerce").astype("Int64")

# (Optional) keep only merge + useful cols from comp
keep_cols = ["cusip","iymd","breach_type","breach_org_name","compustat_gvkey","compustat_tic","compustat_name"]
comp_small = comp[keep_cols].copy()

# Merge
merged = pd.merge(comp_small, fin, on=["cusip","iymd"], how="inner")

# Quick checks
print("comp rows:", comp_small.shape[0], " | fin rows:", fin.shape[0], " | merged rows:", merged.shape[0])
merged.head()


comp rows: 7694  | fin rows: 3045  | merged rows: 4646


,cusip,iymd,breach_type,breach_org_name,compustat_gvkey,compustat_tic,compustat_name,tvol,beta,ivol,...,high_tech,roa,paststkret,IndustryGroup,tvol_post,beta_post,ivol_post,tvol_ch,beta_ch,ivol_ch
0,50212V10,20110923,PHYS,LPL Financial,178519,LPLA,LPL Financial Holdings Inc,0.263265,0.791707,0.214242,...,0,0.044646,NaN,11,0.286502,0.900377,0.230526,0.023237,0.108670,0.016284
1,59522J10,20150520,INSD,Mid-America Apartment Communities,29649,MAA,Mid-America Apartment Communities Inc,0.175000,0.611064,0.155915,...,0,0.048525,0.195557,11,0.207328,0.633612,0.169949,0.032328,0.022548,0.014033
2,59522J10,20150520,INSD,"Mid-America Apartment Communities, Inc.",29649,MAA,Mid-America Apartment Communities Inc,0.175000,0.611064,0.155915,...,0,0.048525,0.195557,11,0.207328,0.633612,0.169949,0.032328,0.022548,0.014033
3,59522J10,20150520,INSD,"Mid-America Apartment Communities, Inc.",29649,MAA,Mid-America Apartment Communities Inc,0.175000,0.611064,0.155915,...,0,0.048525,0.195557,11,0.207328,0.633612,0.169949,0.032328,0.022548,0.014033
4,35671D85,20230908,HACK,Freeport-McMoRan Inc.,14590,FCX,Freeport-McMoRan Inc,0.426244,1.396849,0.327913,...,0,0.035196,0.019929,12,0.257997,1.484862,0.194638,-0.168247,0.088013,-0.133275


In [3]:
import pandas as pd

# --- 1) Industry code -> name mapping (adjust if your mapping differs) ---
ind_map = {
    1: "Consumer Non-Durables",
    2: "Consumer Durables",
    3: "Manufacturing",
    4: "Energy",
    5: "Chemicals",
    6: "Business Equipment (Tech)",
    7: "Telecom/TV",
    8: "Utilities",
    9: "Wholesale/Retail/Services",
    10: "Health Care",
    11: "Finance",
    12: "Other"
}

# --- 2) Clean/select columns ---
df = merged.copy()
df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
df = df.dropna(subset=["year", "IndustryGroup"]).copy()
df["year"] = df["year"].astype(int)
df["IndustryName"] = df["IndustryGroup"].map(ind_map).fillna("Unknown")

# --- 3) Build counts table (Year x Industry) ---
counts = pd.crosstab(df["year"], df["IndustryName"]).sort_index()

# --- 4) Add row totals ---
counts["Total"] = counts.sum(axis=1)

# --- 5) Convert to one table with "count (row%)" in each cell ---
row_totals = counts["Total"]
pct = counts.div(row_totals, axis=0) * 100
formatted = counts.copy().astype(str)

for c in counts.columns:
    formatted[c] = counts[c].astype(int).astype(str) + " (" + pct[c].round(2).astype(str) + "%)"

# --- 6) Add a final Total row (column totals) ---
col_totals = counts.drop(columns="Total", errors="ignore").sum(axis=0)
final_total = counts["Total"].sum()

total_row = {}
for c in counts.columns:
    if c == "Total":
        total_row[c] = f"{final_total} (100.0%)"
    else:
        ct = int(col_totals.get(c, 0))
        pr = 100.0 * ct / final_total if final_total else 0.0
        total_row[c] = f"{ct} ({pr:.2f}%)"

formatted.loc["Total"] = total_row

# --- 7) Save to Excel (change path if you like) ---
out_path = "/Users/amarachukwuobi/Desktop/DataBreach/cyber_attack_distribution_by_year_industry.xlsx"
with pd.ExcelWriter(out_path, engine="xlsxwriter") as w:
    formatted.to_excel(w, sheet_name="Year_by_Industry", index=True)

print("Saved:", out_path)

# Show the first few lines in notebook
formatted.head()


Saved: /Users/amarachukwuobi/Desktop/DataBreach/cyber_attack_distribution_by_year_industry.xlsx


IndustryName,Business Equipment (Tech),Chemicals,Consumer Durables,Consumer Non-Durables,Energy,Finance,Health Care,Manufacturing,Other,Telecom/TV,Utilities,Wholesale/Retail/Services,Total
year,,,,,,,,,,,,,
2011,13 (13.0%),0 (0.0%),0 (0.0%),2 (2.0%),0 (0.0%),65 (65.0%),3 (3.0%),1 (1.0%),10 (10.0%),0 (0.0%),0 (0.0%),6 (6.0%),100 (100.0%)
2012,14 (5.67%),0 (0.0%),0 (0.0%),2 (0.81%),0 (0.0%),206 (83.4%),7 (2.83%),3 (1.21%),7 (2.83%),3 (1.21%),0 (0.0%),5 (2.02%),247 (100.0%)
2013,8 (3.07%),0 (0.0%),0 (0.0%),1 (0.38%),0 (0.0%),200 (76.63%),13 (4.98%),8 (3.07%),13 (4.98%),1 (0.38%),0 (0.0%),17 (6.51%),261 (100.0%)
2014,15 (5.77%),1 (0.38%),0 (0.0%),10 (3.85%),4 (1.54%),179 (68.85%),8 (3.08%),1 (0.38%),7 (2.69%),10 (3.85%),6 (2.31%),19 (7.31%),260 (100.0%)
2015,38 (11.55%),2 (0.61%),1 (0.3%),0 (0.0%),1 (0.3%),191 (58.05%),15 (4.56%),6 (1.82%),28 (8.51%),6 (1.82%),0 (0.0%),41 (12.46%),329 (100.0%)


In [7]:
# Create a normalized breach type variable if missing
def norm_btype(x):
    if pd.isna(x): return "UNKN"
    s = str(x).upper()
    if "CARD" in s or "PAYMENT" in s: return "CARD"
    if "HACK" in s or "MALWARE" in s: return "HACK"
    if "INS"  in s: return "INSD"
    if "PHYS" in s: return "PHYS"
    if "PORT" in s and "REPORT" not in s: return "PORT"
    if "STAT" in s and "STATION" in s: return "STAT"
    if "DISC" in s or "DISCLOS" in s or "UNINTENDED" in s: return "DISC"
    if "UNK"  in s: return "UNKN"
    return "UNKN"

# If breach_type exists, normalize it
if "breach_type" in merged.columns:
    merged["breach_type_norm"] = merged["breach_type"].map(norm_btype)
else:
    # fallback: no breach info in this dataset
    merged["breach_type_norm"] = "UNKN"


In [9]:
# Counts
print("Attacked:", (merged["breach_type_norm"]!="UNKN").sum())
print("Non-attacked:", (merged["breach_type_norm"]=="UNKN").sum())

# Preview your summary table if you ran the last cell:
table.head()  # should show the A vs B means/medians and stars

Attacked: 1928
Non-attacked: 2718


NameError: name 'table' is not defined

In [11]:
import pandas as pd
import numpy as np
from scipy import stats as st

# Split attacked vs non-attacked
attacked = merged[merged["breach_type_norm"] != "UNKN"]
non_attacked = merged[merged["breach_type_norm"] == "UNKN"]

print(f"Attacked firm-years: {len(attacked)}")
print(f"Non-attacked firm-years: {len(non_attacked)}")

# Variables to include (filter only those that exist in your dataset)
candidates = ["asset","mtb","roa","salegrow_yr","bklev","frmlev","dpo","fcf","ppe","noncash_workcap"]
vars_used = [v for v in candidates if v in merged.columns]

# Helper: stars for p-values
def stars(p):
    return "***" if p < 0.01 else "**" if p < 0.05 else "*" if p < 0.10 else ""

rows = []
for v in vars_used:
    a = pd.to_numeric(attacked[v], errors="coerce").dropna()
    b = pd.to_numeric(non_attacked[v], errors="coerce").dropna()
    if len(a)==0 or len(b)==0:
        continue
    
    # Means / Medians
    mean_a, mean_b = a.mean(), b.mean()
    med_a,  med_b  = a.median(), b.median()

    # t-test (means)
    tstat, p_mean = st.ttest_ind(a, b, equal_var=False, nan_policy="omit")
    diff_mean_fmt = f"{(mean_a-mean_b):.3f}{stars(p_mean)}"

    # Mann-Whitney U (medians)
    ustat, p_med = st.mannwhitneyu(a, b, alternative="two-sided")
    diff_med_fmt = f"{(med_a-med_b):.3f}{stars(p_med)}"

    rows.append([
        v,
        len(a), round(mean_a,3), round(med_a,3),
        len(b), round(mean_b,3), round(med_b,3),
        diff_mean_fmt, diff_med_fmt
    ])

table = pd.DataFrame(rows, columns=[
    "Variable",
    "N (A)", "Mean (A)", "Median (A)",
    "N (B)", "Mean (B)", "Median (B)",
    "A–B (Mean)", "A–B (Median)"
])

# Show table in notebook
display(table)

# Save to Excel
out_path = "/Users/amarachukwuobi/Desktop/DataBreach/attacked_vs_nonattacked_summary.xlsx"
table.to_excel(out_path, index=False)

print(f"✅ Summary table saved to: {out_path}")
print("Stars: *** p<0.01, ** p<0.05, * p<0.10")


Attacked firm-years: 1928
Non-attacked firm-years: 2718


,Variable,N (A),Mean (A),Median (A),N (B),Mean (B),Median (B),A–B (Mean),A–B (Median)
0,asset,1928,140145.100,11174.876,2718,176886.625,32132.000,-36741.524**,-20957.124***
1,mtb,1928,1.442,1.015,2718,1.788,0.947,-0.346***,0.068
2,roa,1741,0.022,0.027,2509,0.037,0.032,-0.015***,-0.005***
3,salegrow_yr,1737,0.222,0.066,2506,0.152,0.047,0.069,0.019***
4,bklev,1928,0.301,0.260,2718,0.296,0.285,0.005,-0.025**
5,frmlev,1741,1.076,0.308,2509,0.802,0.552,0.275***,-0.244***
6,dpo,1928,0.126,0.078,2718,0.120,0.106,0.006,-0.028***
7,fcf,1928,0.024,0.036,2718,0.036,0.032,-0.012***,0.004*
8,ppe,1872,0.170,0.081,2662,0.130,0.036,0.039***,0.045***
9,noncash_workcap,1928,0.644,0.698,2718,0.710,0.821,-0.065***,-0.123***


✅ Summary table saved to: /Users/amarachukwuobi/Desktop/DataBreach/attacked_vs_nonattacked_summary.xlsx
Stars: *** p<0.01, ** p<0.05, * p<0.10


In [17]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# ====== CONFIG ======
out_xlsx = "/Users/amarachukwuobi/Desktop/DataBreach/analysis_core_outputs.xlsx"
out_txt  = "/Users/amarachukwuobi/Desktop/DataBreach/ols_summaries.txt"

# ====== (A) CLEAN NUMERIC FIELDS ======
corr_vars = ["tvol_ch","beta_ch","ivol_ch"]
stats_vars = ["tvol_ch","beta_ch","ivol_ch","bklev","fcf","mtb",
              "noncash_workcap","salegrow_yr","asset","ppe","frmlev","dpo","roa"]
for c in set(corr_vars + stats_vars):
    if c in merged.columns:
        merged[c] = pd.to_numeric(merged[c], errors="coerce")

# Ensure FE fields exist
if "year" not in merged.columns and "iymd" in merged.columns:
    merged["year"] = (pd.to_numeric(merged["iymd"], errors="coerce")//10000).astype("Int64")
if "IndustryGroup" not in merged.columns:
    merged["IndustryGroup"] = "Unknown"

# ====== (5) CORRELATION MATRIX ======
corr = merged[corr_vars].corr()

# ====== (6) SUMMARY STATISTICS ======
sub = merged[stats_vars]
summary_stats = sub.describe().T
summary_stats["missing"] = len(sub) - sub.notna().sum()

# ====== (8) OLS (no FE) ======
predictors = ["high_tech","bklev","fcf","mtb","noncash_workcap",
              "salegrow_yr","asset","ppe","frmlev","dpo","roa"]
# coerce predictors numeric
for c in predictors:
    if c in merged.columns:
        merged[c] = pd.to_numeric(merged[c], errors="coerce")

def run_ols(dep, fe=False):
    rhs = " + ".join([p for p in predictors if p in merged.columns])
    if fe:
        rhs += " + C(IndustryGroup) + C(year)"
    formula = f"{dep} ~ {rhs}"
    cols_needed = [dep] + [p for p in predictors if p in merged.columns] + (["IndustryGroup","year"] if fe else [])
    data = merged[cols_needed].dropna()
    if data.empty:
        print(f"[WARN] No data for model: {formula}")
        return None
    model = smf.ols(formula, data=data).fit(cov_type="HC3")  # robust SE
    print(f"\n=== OLS {'(with FE)' if fe else ''} for {dep} ===\n{model.summary()}")
    return model

m_tvol   = run_ols("tvol_ch", fe=False)
m_beta   = run_ols("beta_ch", fe=False)
m_ivol   = run_ols("ivol_ch", fe=False)

# ====== (9) OLS with year & industry FE ======
m_tvol_fe = run_ols("tvol_ch", fe=True)
m_beta_fe = run_ols("beta_ch", fe=True)
m_ivol_fe = run_ols("ivol_ch", fe=True)

# ====== SAVE TABLES ======
with pd.ExcelWriter(out_xlsx, engine="xlsxwriter") as w:
    corr.to_excel(w, sheet_name="corr_matrix")
    summary_stats.to_excel(w, sheet_name="summary_stats")

print(f"\nSaved tables to: {out_xlsx}")

# ====== SAVE REGRESSION SUMMARIES (TEXT) ======
with open(out_txt, "w") as f:
    for tag, m in [
        ("tvol_ch (no FE)", m_tvol),
        ("beta_ch (no FE)", m_beta),
        ("ivol_ch (no FE)", m_ivol),
        ("tvol_ch (with FE)", m_tvol_fe),
        ("beta_ch (with FE)", m_beta_fe),
        ("ivol_ch (with FE)", m_ivol_fe),
    ]:
        if m is not None:
            f.write(f"=== {tag} ===\n")
            f.write(m.summary().as_text())
            f.write("\n\n")
print(f"Saved regression summaries to: {out_txt}")


/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 11, but rank is 10
  warnings.warn('covariance of constraints does not have full '
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 11, but rank is 10
  warnings.warn('covariance of constraints does not have full '
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 11, but rank is 10
  warnings.warn('covariance of constraints does not have full '
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 35, but rank is 34
  warnings.warn('covariance of constraints does not 


=== OLS  for tvol_ch ===
                            OLS Regression Results                            
Dep. Variable:                tvol_ch   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     6.527
Date:                Thu, 21 Aug 2025   Prob (F-statistic):           4.44e-10
Time:                        22:51:37   Log-Likelihood:                 1012.3
No. Observations:                3866   AIC:                            -2001.
Df Residuals:                    3854   BIC:                            -1926.
Df Model:                          11                                         
Covariance Type:                  HC3                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept       